In [14]:
from constants import PATH_JSON_ANNOTATIONS, PATH_JSON_QUESTIONS, OPENAI_KEY, PROMPT_CONSTRUCAO_TEXTO, PROMPT_MIX_RESPONSE, PROMPT_SANITY_CHECK
from utils import load_json, get_image_id, encode_image
import os
import pandas as pd
import openai
from openai import OpenAI
import base64
from collections import Counter
import re
import json
import time

In [2]:
df = pd.read_csv('data/final_data_descriptions.csv')

In [3]:
df

,Unnamed: 0.1,Unnamed: 0,image_path,image_id,question,answer,Tm,Tc,Ans,Ti
0,0,0,COCO_val2014_000000000042.jpg,42,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,red,"Several days after the low dissipated , the r..."
1,1,1,COCO_val2014_000000000073.jpg,73,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,red,Kawaguchi 's Center Body of troops was planni...
2,2,2,COCO_val2014_000000000074.jpg,74,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,red,"In the centre , the main attack along the Bui..."
3,3,3,COCO_val2014_000000000133.jpg,133,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,red,Mount Elbert was named by miners in honor of ...
4,4,4,COCO_val2014_000000000136.jpg,136,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,red,"Runs west through Jackson , Mississippi , eve..."
...,...,...,...,...,...,...,...,...,...,...
991,991,991,COCO_val2014_000000014135.jpg,14135,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,no,The documentary film Tim Richmond : To The Li...
992,992,992,COCO_val2014_000000014151.jpg,14151,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,"No, this is not at the Olympics.",Kevin Spacey as David <unk> \n
993,993,993,COCO_val2014_000000014167.jpg,14167,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,blue,The Island Def Jam rapper Big K.R.I.T. was bo...
994,994,994,COCO_val2014_000000014175.jpg,14175,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,red,"A total of 2 @,@ 000 people attended Slammive..."


In [4]:
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

## Generate Answers Functions

In [5]:
def generate_answers_text_only(text_col: str, df: pd.DataFrame, name_model: str, model):

    answers = []
    model_df = df.copy()

    for i in model_df.index:
        image_infos = model_df.iloc[i]
        image_description = image_infos[text_col]
        image_question = image_infos['question']

        prompt_formatado = PROMPT_SANITY_CHECK.format(text=image_description, question=image_question)

        print(f"Gerando resposta para a imagem {i}.")

        response = model.chat.completions.create(
            model=name_model,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt_formatado
                        }
                    ]
                }
            ], temperature=0.0, timeout=20.0
        )
        texts = response.choices[0].message.content
        answers.append(texts)

        print(f"Textos gerados: {texts}")

    return answers

In [21]:
def generate_answers_image_only(df: pd.DataFrame, name_model: str, model):

    answers = []
    base_image_path = 'images/'

    for i in range(df.shape[0]):

        image_infos = df.iloc[i]
        image_question = image_infos['question']
        image_path = base_image_path + image_infos['image_path']
        
        encoded_image = encode_image(image_path)

        print(f"Gerando textos para a imagem {i}.")

        response = model.chat.completions.create(
            model=name_model,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url" : {"url": f"data:image/jpeg;base64,{encoded_image}"}
                        },
                        {
                            "type": "text",
                            "text": f"{image_question} Respond only with the final answer, without explanation or full sentences."
                        }
                    ]
                }
            ], temperature=0.0, timeout=20.0
        )
        texts = response.choices[0].message.content
        answers.append(texts)

        time.sleep(1)

        print(f"Textos gerados: {texts}")

    return answers


In [ ]:
def generate_answers_mix(text_col: str, df: pd.DataFrame, name_model: str, model):

    answers = []
    model_df = df.copy()
    base_image_path = 'images/'

    for i in model_df.index:
        image_infos = model_df.iloc[i]
        image_description = image_infos[text_col]
        image_question = image_infos['question']
        image_path = base_image_path + image_infos['image_path']
        
        encoded_image = encode_image(image_path)

        prompt_formatado = PROMPT_MIX_RESPONSE.format(text_information=image_description, question=image_question)

        print(f"Gerando resposta para a imagem {i}.")

        response = model.chat.completions.create(
            model=name_model,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url" : {"url": f"data:image/jpeg;base64,{encoded_image}"}
                        },
                        {
                            "type": "text",
                            "text": prompt_formatado
                        }
                    ]
                }
            ], temperature=0.0, timeout=20.0
        )
        texts = response.choices[0].message.content
        answers.append(texts)

        time.sleep(1)

        print(f"Textos gerados: {texts}")

    return answers

## GPT-4o Mini

In [8]:
mini_df = df[['image_path', 'question', 'answer', 'Tm', 'Tc', 'Ti']]

In [9]:
mini_df

,image_path,question,answer,Tm,Tc,Ti
0,COCO_val2014_000000000042.jpg,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,"Several days after the low dissipated , the r..."
1,COCO_val2014_000000000073.jpg,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,Kawaguchi 's Center Body of troops was planni...
2,COCO_val2014_000000000074.jpg,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,"In the centre , the main attack along the Bui..."
3,COCO_val2014_000000000133.jpg,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,Mount Elbert was named by miners in honor of ...
4,COCO_val2014_000000000136.jpg,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,"Runs west through Jackson , Mississippi , eve..."
...,...,...,...,...,...,...
991,COCO_val2014_000000014135.jpg,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,The documentary film Tim Richmond : To The Li...
992,COCO_val2014_000000014151.jpg,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,Kevin Spacey as David <unk> \n
993,COCO_val2014_000000014167.jpg,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,The Island Def Jam rapper Big K.R.I.T. was bo...
994,COCO_val2014_000000014175.jpg,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,"A total of 2 @,@ 000 people attended Slammive..."


In [10]:
mini_model = OpenAI(api_key=OPENAI_KEY)

In [18]:
mini_tm_responses = generate_answers_text_only('Tm', mini_df, 'gpt-4o-mini', mini_model)

Gerando resposta para a imagem 0.
Textos gerados: Blue
Gerando resposta para a imagem 1.
Textos gerados: SV-6260
Gerando resposta para a imagem 2.
Textos gerados: Yes.
Gerando resposta para a imagem 3.
Textos gerados: Blue
Gerando resposta para a imagem 4.
Textos gerados: No.
Gerando resposta para a imagem 5.
Textos gerados: Looking outside.
Gerando resposta para a imagem 6.
Textos gerados: Yellow
Gerando resposta para a imagem 7.
Textos gerados: One
Gerando resposta para a imagem 8.
Textos gerados: Baseball
Gerando resposta para a imagem 9.
Textos gerados: Corn
Gerando resposta para a imagem 10.
Textos gerados: Turn the faucet handle with the blue color.
Gerando resposta para a imagem 11.
Textos gerados: He is standing on a white balance board.
Gerando resposta para a imagem 12.
Textos gerados: There is no car mentioned; the truck on the far right is red.
Gerando resposta para a imagem 13.
Textos gerados: Cupcake
Gerando resposta para a imagem 14.
Textos gerados: Yes.
Gerando resposta

In [19]:
mini_tc_responses = generate_answers_text_only('Tc', mini_df, 'gpt-4o-mini', mini_model)

Gerando resposta para a imagem 0.
Textos gerados: Red
Gerando resposta para a imagem 1.
Textos gerados: AB-1234
Gerando resposta para a imagem 2.
Textos gerados: Yes.
Gerando resposta para a imagem 3.
Textos gerados: Red
Gerando resposta para a imagem 4.
Textos gerados: No.
Gerando resposta para a imagem 5.
Textos gerados: Reading a book.
Gerando resposta para a imagem 6.
Textos gerados: Red
Gerando resposta para a imagem 7.
Textos gerados: Three
Gerando resposta para a imagem 8.
Textos gerados: Soccer
Gerando resposta para a imagem 9.
Textos gerados: Lemons
Gerando resposta para a imagem 10.
Textos gerados: Turn the red handle.
Gerando resposta para a imagem 11.
Textos gerados: A blue chair
Gerando resposta para a imagem 12.
Textos gerados: There is no car mentioned; the truck on the far right is green.
Gerando resposta para a imagem 13.
Textos gerados: Sunset
Gerando resposta para a imagem 14.
Textos gerados: No.
Gerando resposta para a imagem 15.
Textos gerados: No.
Gerando resposta

In [23]:
mini_ti_responses = generate_answers_text_only('Ti', mini_df, 'gpt-4o-mini', mini_model)

Gerando resposta para a imagem 0.
Textos gerados: The text does not provide information about the color of the gym shoes.
Gerando resposta para a imagem 1.
Textos gerados: The text does not provide a license number.
Gerando resposta para a imagem 2.
Textos gerados: The text does not provide information about a dog or a collar.
Gerando resposta para a imagem 3.
Textos gerados: The text does not provide information about the color of a lamp.
Gerando resposta para a imagem 4.
Textos gerados: No.
Gerando resposta para a imagem 5.
Textos gerados: The text does not mention a woman in the room.
Gerando resposta para a imagem 6.
Textos gerados: The text does not provide information about the color of the birds' tails.
Gerando resposta para a imagem 7.
Textos gerados: The text does not provide information about paper towel rolls.
Gerando resposta para a imagem 8.
Textos gerados: The text does not mention any sport being played.
Gerando resposta para a imagem 9.
Textos gerados: Corn
Gerando resp

In [27]:
mini_df['Tc_responses'] = pd.Series(mini_tc_responses)
mini_df['Tm_responses'] = pd.Series(mini_tm_responses)
mini_df['Ti_responses'] = pd.Series(mini_ti_responses)

C:\Users\gabri\AppData\Local\Temp\ipykernel_23328\2761496808.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_df['Tc_responses'] = pd.Series(mini_tc_responses)


In [28]:
mini_df

,image_path,question,answer,Tm,Tc,Ti,Tc_responses,Tm_responses,Ti_responses
0,COCO_val2014_000000000042.jpg,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,"Several days after the low dissipated , the r...",Red,Blue,The text does not provide information about th...
1,COCO_val2014_000000000073.jpg,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,Kawaguchi 's Center Body of troops was planni...,AB-1234,SV-6260,The text does not provide a license number.
2,COCO_val2014_000000000074.jpg,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,"In the centre , the main attack along the Bui...",Yes.,Yes.,The text does not provide information about a ...
3,COCO_val2014_000000000133.jpg,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,Mount Elbert was named by miners in honor of ...,Red,Blue,The text does not provide information about th...
4,COCO_val2014_000000000136.jpg,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,"Runs west through Jackson , Mississippi , eve...",No.,No.,No.
...,...,...,...,...,...,...,...,...,...
991,COCO_val2014_000000014135.jpg,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,The documentary film Tim Richmond : To The Li...,No.,Yes.,The text does not provide information about wh...
992,COCO_val2014_000000014151.jpg,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,Kevin Spacey as David <unk> \n,Not enough information.,Yes.,No.
993,COCO_val2014_000000014167.jpg,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,The Island Def Jam rapper Big K.R.I.T. was bo...,Yes.,No.,The text does not provide information about a ...
994,COCO_val2014_000000014175.jpg,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,"A total of 2 @,@ 000 people attended Slammive...",A red object.,A blue object.,The text does not provide information about a ...


In [29]:
mini_df.to_csv('data/gpt_4o_mini_responses.csv')

In [17]:
mini_only_image_responses = generate_answers_image_only(mini_df, 'gpt-4o-mini', mini_model)

Gerando textos para a imagem 0.
Textos gerados: White.
Gerando textos para a imagem 1.
Textos gerados: SV-6260
Gerando textos para a imagem 2.
Textos gerados: No.
Gerando textos para a imagem 3.
Textos gerados: Blue.
Gerando textos para a imagem 4.
Textos gerados: No.
Gerando textos para a imagem 5.
Textos gerados: The woman is standing near the kitchen, likely preparing food or doing something related to cooking.
Gerando textos para a imagem 6.
Textos gerados: Yellow.
Gerando textos para a imagem 7.
Textos gerados: One.
Gerando textos para a imagem 8.
Textos gerados: Baseball
Gerando textos para a imagem 9.
Textos gerados: Corn.
Gerando textos para a imagem 10.
Textos gerados: Turn the faucet handle to the right.
Gerando textos para a imagem 11.
Textos gerados: He is standing on a balance board.
Gerando textos para a imagem 12.
Textos gerados: Red
Gerando textos para a imagem 13.
Textos gerados: Cupcake
Gerando textos para a imagem 14.
Textos gerados: Yes.
Gerando textos para a imagem

In [26]:
mini_df = pd.read_csv('data/gpt_4o_mini_responses.csv')

In [28]:
mini_df['Base_responses'] = pd.Series(mini_only_image_responses)

In [29]:
mini_df

,Unnamed: 0,image_path,question,answer,Tm,Tc,Ti,Tc_responses,Tm_responses,Ti_responses,Base_responses
0,0,COCO_val2014_000000000042.jpg,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,"Several days after the low dissipated , the r...",Red,Blue,The text does not provide information about th...,White.
1,1,COCO_val2014_000000000073.jpg,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,Kawaguchi 's Center Body of troops was planni...,AB-1234,SV-6260,The text does not provide a license number.,SV-6260
2,2,COCO_val2014_000000000074.jpg,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,"In the centre , the main attack along the Bui...",Yes.,Yes.,The text does not provide information about a ...,No.
3,3,COCO_val2014_000000000133.jpg,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,Mount Elbert was named by miners in honor of ...,Red,Blue,The text does not provide information about th...,Blue.
4,4,COCO_val2014_000000000136.jpg,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,"Runs west through Jackson , Mississippi , eve...",No.,No.,No.,No.
...,...,...,...,...,...,...,...,...,...,...,...
991,991,COCO_val2014_000000014135.jpg,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,The documentary film Tim Richmond : To The Li...,No.,Yes.,The text does not provide information about wh...,Yes.
992,992,COCO_val2014_000000014151.jpg,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,Kevin Spacey as David <unk> \n,Not enough information.,Yes.,No.,Yes.
993,993,COCO_val2014_000000014167.jpg,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,The Island Def Jam rapper Big K.R.I.T. was bo...,Yes.,No.,The text does not provide information about a ...,No.
994,994,COCO_val2014_000000014175.jpg,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,"A total of 2 @,@ 000 people attended Slammive...",A red object.,A blue object.,The text does not provide information about a ...,A nozzle.


In [30]:
mini_df.to_csv('data/gpt_4o_mini_responses.csv')

In [32]:
mini_mix_tm_responses = generate_answers_mix('Tm',mini_df, 'gpt-4o-mini', mini_model)

Gerando resposta para a imagem 0.
Textos gerados: Blue
Gerando resposta para a imagem 1.
Textos gerados: SV-6260
Gerando resposta para a imagem 2.
Textos gerados: Yes.
Gerando resposta para a imagem 3.
Textos gerados: Blue
Gerando resposta para a imagem 4.
Textos gerados: No.
Gerando resposta para a imagem 5.
Textos gerados: Looking outside.
Gerando resposta para a imagem 6.
Textos gerados: Yellow
Gerando resposta para a imagem 7.
Textos gerados: One.
Gerando resposta para a imagem 8.
Textos gerados: Baseball
Gerando resposta para a imagem 9.
Textos gerados: Corn
Gerando resposta para a imagem 10.
Textos gerados: Turn the faucet handle to the blue side.
Gerando resposta para a imagem 11.
Textos gerados: He is standing on a white balance board.
Gerando resposta para a imagem 12.
Textos gerados: Red.
Gerando resposta para a imagem 13.
Textos gerados: Cupcake
Gerando resposta para a imagem 14.
Textos gerados: Yes.
Gerando resposta para a imagem 15.
Textos gerados: Yes.
Gerando resposta pa

In [33]:
mini_df['mix_tm_responses'] = pd.Series(mini_mix_tm_responses)

In [34]:
mini_df

,Unnamed: 0,image_path,question,answer,Tm,Tc,Ti,Tc_responses,Tm_responses,Ti_responses,Base_responses,mix_tm_responses
0,0,COCO_val2014_000000000042.jpg,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,"Several days after the low dissipated , the r...",Red,Blue,The text does not provide information about th...,White.,Blue
1,1,COCO_val2014_000000000073.jpg,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,Kawaguchi 's Center Body of troops was planni...,AB-1234,SV-6260,The text does not provide a license number.,SV-6260,SV-6260
2,2,COCO_val2014_000000000074.jpg,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,"In the centre , the main attack along the Bui...",Yes.,Yes.,The text does not provide information about a ...,No.,Yes.
3,3,COCO_val2014_000000000133.jpg,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,Mount Elbert was named by miners in honor of ...,Red,Blue,The text does not provide information about th...,Blue.,Blue
4,4,COCO_val2014_000000000136.jpg,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,"Runs west through Jackson , Mississippi , eve...",No.,No.,No.,No.,No.
...,...,...,...,...,...,...,...,...,...,...,...,...
991,991,COCO_val2014_000000014135.jpg,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,The documentary film Tim Richmond : To The Li...,No.,Yes.,The text does not provide information about wh...,Yes.,Yes.
992,992,COCO_val2014_000000014151.jpg,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,Kevin Spacey as David <unk> \n,Not enough information.,Yes.,No.,Yes.,Yes.
993,993,COCO_val2014_000000014167.jpg,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,The Island Def Jam rapper Big K.R.I.T. was bo...,Yes.,No.,The text does not provide information about a ...,No.,No.
994,994,COCO_val2014_000000014175.jpg,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,"A total of 2 @,@ 000 people attended Slammive...",A red object.,A blue object.,The text does not provide information about a ...,A nozzle.,A blue object.


In [35]:
mini_df.to_csv('data/gpt_4o_mini_responses.csv')

In [37]:
mini_mix_tc_responses = generate_answers_mix('Tc',mini_df, 'gpt-4o-mini', mini_model)

Gerando resposta para a imagem 0.
Textos gerados: Red
Gerando resposta para a imagem 1.
Textos gerados: SV-6260
Gerando resposta para a imagem 2.
Textos gerados: Yes.
Gerando resposta para a imagem 3.
Textos gerados: Red
Gerando resposta para a imagem 4.
Textos gerados: No.
Gerando resposta para a imagem 5.
Textos gerados: The woman is standing in the kitchen.
Gerando resposta para a imagem 6.
Textos gerados: Red
Gerando resposta para a imagem 7.
Textos gerados: Three.
Gerando resposta para a imagem 8.
Textos gerados: Baseball
Gerando resposta para a imagem 9.
Textos gerados: Corn.
Gerando resposta para a imagem 10.
Textos gerados: Turn the faucet handle to the right.
Gerando resposta para a imagem 11.
Textos gerados: He is standing on a balance board.
Gerando resposta para a imagem 12.
Textos gerados: Red.
Gerando resposta para a imagem 13.
Textos gerados: Cupcake
Gerando resposta para a imagem 14.
Textos gerados: No.
Gerando resposta para a imagem 15.
Textos gerados: No.
Gerando resp

In [38]:
mini_df['mix_tc_responses'] = pd.Series(mini_mix_tc_responses)

In [39]:
mini_df.to_csv('data/gpt_4o_mini_responses.csv')

In [41]:
mini_df

,Unnamed: 0,image_path,question,answer,Tm,Tc,Ti,Tc_responses,Tm_responses,Ti_responses,Base_responses,mix_tm_responses,mix_tc_responses
0,0,COCO_val2014_000000000042.jpg,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,"Several days after the low dissipated , the r...",Red,Blue,The text does not provide information about th...,White.,Blue,Red
1,1,COCO_val2014_000000000073.jpg,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,Kawaguchi 's Center Body of troops was planni...,AB-1234,SV-6260,The text does not provide a license number.,SV-6260,SV-6260,SV-6260
2,2,COCO_val2014_000000000074.jpg,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,"In the centre , the main attack along the Bui...",Yes.,Yes.,The text does not provide information about a ...,No.,Yes.,Yes.
3,3,COCO_val2014_000000000133.jpg,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,Mount Elbert was named by miners in honor of ...,Red,Blue,The text does not provide information about th...,Blue.,Blue,Red
4,4,COCO_val2014_000000000136.jpg,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,"Runs west through Jackson , Mississippi , eve...",No.,No.,No.,No.,No.,No.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,991,COCO_val2014_000000014135.jpg,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,The documentary film Tim Richmond : To The Li...,No.,Yes.,The text does not provide information about wh...,Yes.,Yes.,No.
992,992,COCO_val2014_000000014151.jpg,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,Kevin Spacey as David <unk> \n,Not enough information.,Yes.,No.,Yes.,Yes.,Yes.
993,993,COCO_val2014_000000014167.jpg,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,The Island Def Jam rapper Big K.R.I.T. was bo...,Yes.,No.,The text does not provide information about a ...,No.,No.,Yes.
994,994,COCO_val2014_000000014175.jpg,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,"A total of 2 @,@ 000 people attended Slammive...",A red object.,A blue object.,The text does not provide information about a ...,A nozzle.,A blue object.,A red object.


In [42]:
mini_mix_ti_responses = generate_answers_mix('Ti',mini_df, 'gpt-4o-mini', mini_model)

Gerando resposta para a imagem 0.
Textos gerados: White.
Gerando resposta para a imagem 1.
Textos gerados: SV-6260
Gerando resposta para a imagem 2.
Textos gerados: I can't tell if the dog has a collar.
Gerando resposta para a imagem 3.
Textos gerados: Blue
Gerando resposta para a imagem 4.
Textos gerados: Yes.
Gerando resposta para a imagem 5.
Textos gerados: The woman is standing near the kitchen area.
Gerando resposta para a imagem 6.
Textos gerados: The tips of the birds' tails are not visible in the image.
Gerando resposta para a imagem 7.
Textos gerados: One.
Gerando resposta para a imagem 8.
Textos gerados: Baseball
Gerando resposta para a imagem 9.
Textos gerados: Corn.
Gerando resposta para a imagem 10.
Textos gerados: Turn the faucet handle counterclockwise.
Gerando resposta para a imagem 11.
Textos gerados: He is not sitting; he is standing on a balance board.
Gerando resposta para a imagem 12.
Textos gerados: Red.
Gerando resposta para a imagem 13.
Textos gerados: Cupcake
G

In [43]:
mini_df['mix_ti_responses'] = pd.Series(mini_mix_tc_responses)

In [44]:
mini_df

,Unnamed: 0,image_path,question,answer,Tm,Tc,Ti,Tc_responses,Tm_responses,Ti_responses,Base_responses,mix_tm_responses,mix_tc_responses,mix_ti_responses
0,0,COCO_val2014_000000000042.jpg,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,"Several days after the low dissipated , the r...",Red,Blue,The text does not provide information about th...,White.,Blue,Red,Red
1,1,COCO_val2014_000000000073.jpg,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,Kawaguchi 's Center Body of troops was planni...,AB-1234,SV-6260,The text does not provide a license number.,SV-6260,SV-6260,SV-6260,SV-6260
2,2,COCO_val2014_000000000074.jpg,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,"In the centre , the main attack along the Bui...",Yes.,Yes.,The text does not provide information about a ...,No.,Yes.,Yes.,Yes.
3,3,COCO_val2014_000000000133.jpg,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,Mount Elbert was named by miners in honor of ...,Red,Blue,The text does not provide information about th...,Blue.,Blue,Red,Red
4,4,COCO_val2014_000000000136.jpg,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,"Runs west through Jackson , Mississippi , eve...",No.,No.,No.,No.,No.,No.,No.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,991,COCO_val2014_000000014135.jpg,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,The documentary film Tim Richmond : To The Li...,No.,Yes.,The text does not provide information about wh...,Yes.,Yes.,No.,No.
992,992,COCO_val2014_000000014151.jpg,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,Kevin Spacey as David <unk> \n,Not enough information.,Yes.,No.,Yes.,Yes.,Yes.,Yes.
993,993,COCO_val2014_000000014167.jpg,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,The Island Def Jam rapper Big K.R.I.T. was bo...,Yes.,No.,The text does not provide information about a ...,No.,No.,Yes.,Yes.
994,994,COCO_val2014_000000014175.jpg,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,"A total of 2 @,@ 000 people attended Slammive...",A red object.,A blue object.,The text does not provide information about a ...,A nozzle.,A blue object.,A red object.,A red object.


In [45]:
mini_df = mini_df.drop(columns=['Unnamed: 0'])

In [46]:
mini_df.to_csv('data/gpt_4o_mini_responses.csv')